# Elastic Properties
Calculate the bulk modulus for Aluminium using the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) DFT code:

## Equation of State 
One way to calculate the bulk modulus is using the Equation of State to calculate the equilibrium properties:

In [1]:
from ase.build import bulk
from atomistics.calculators.ase import evaluate_with_ase
from atomistics.workflows.evcurve.workflow import EnergyVolumeCurveWorkflow
from gpaw import GPAW, PW

workflow = EnergyVolumeCurveWorkflow(
    structure=bulk("Al", a=4.05, cubic=True),
    num_points=11,
    fit_type='polynomial',
    fit_order=3,
    vol_range=0.05,
    axes=['x', 'y', 'z'],
    strains=None,
)
task_dict = workflow.generate_structures()
task_dict

{'calc_energy': OrderedDict([(0.95,
               Atoms(symbols='Al4', pbc=True, cell=[3.9813426685908118, 3.9813426685908118, 3.9813426685908118])),
              (0.96,
               Atoms(symbols='Al4', pbc=True, cell=[3.9952635604153612, 3.9952635604153612, 3.9952635604153612])),
              (0.97,
               Atoms(symbols='Al4', pbc=True, cell=[4.009088111958974, 4.009088111958974, 4.009088111958974])),
              (0.98,
               Atoms(symbols='Al4', pbc=True, cell=[4.022817972936038, 4.022817972936038, 4.022817972936038])),
              (0.99,
               Atoms(symbols='Al4', pbc=True, cell=[4.036454748321015, 4.036454748321015, 4.036454748321015])),
              (1.0, Atoms(symbols='Al4', pbc=True, cell=[4.05, 4.05, 4.05])),
              (1.01,
               Atoms(symbols='Al4', pbc=True, cell=[4.063455248345461, 4.063455248345461, 4.063455248345461])),
              (1.02,
               Atoms(symbols='Al4', pbc=True, cell=[4.076821973718458, 4.076821973

In the first step the `EnergyVolumeCurveWorkflow` object is initialized including all the parameters to generate
the strained structures and afterwards fit the resulting energy volume curve. This allows the user to see all relevant
parameters at one place. After the initialization the function `generate_structures()` is called without any
additional parameters. This function returns the task dictionary `task_dict` which includes the tasks which should
be executed by the calculator. In this case the task is to calculate the energy `calc_energy` of the eleven generated 
structures. Each structure is labeled by the ratio of compression or elongation. In the second step the `task_dict` 
is evaluated with the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) simulation code using the `evaluate_with_ase()` function:

In [2]:
result_dict = evaluate_with_ase(
    task_dict=task_dict,
    ase_calculator=GPAW(
        xc="PBE",
        mode=PW(300),
        kpts=(3, 3, 3)
    )
)
result_dict


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  23.9.1
 |___|_|             

User:   janssen@cmpc06
Date:   Wed Nov 15 11:29:13 2023
Arch:   x86_64
Pid:    15086
CWD:    /home/janssen/notebooks/2023/2023-11-15-atomistics-example
Python: 3.10.13
gpaw:   /home/janssen/mambaforge/lib/python3.10/site-packages/gpaw
_gpaw:  /home/janssen/mambaforge/lib/python3.10/site-packages/
        _gpaw.cpython-310-x86_64-linux-gnu.so
ase:    /home/janssen/mambaforge/lib/python3.10/site-packages/ase (version 3.22.1)
numpy:  /home/janssen/mambaforge/lib/python3.10/site-packages/numpy (version 1.26.0)
scipy:  /home/janssen/mambaforge/lib/python3.10/site-packages/scipy (version 1.11.3)
libxc:  6.2.2
units:  Angstrom and eV
cores: 1
OpenMP: True
OMP_NUM_THREADS: 1

Input parameters:
  kpts: [3 3 3]
  mode: {ecut: 300.0,
         name: pw}
  xc: PBE

System changes: positions, numbers, cell, pbc, initial_charges, initial_magmoms 

Initialize ...

species:
  Al:
    n

{'energy': {0.95: -14.895378072823371,
  0.96: -14.910819737657517,
  0.97: -14.922307241120716,
  0.98: -14.930392279319857,
  0.99: -14.935048569966273,
  1.0: -14.93666639636423,
  1.01: -14.935212782128978,
  1.02: -14.931045138840679,
  1.03: -14.924165445707086,
  1.04: -14.914703574004657,
  1.05: -14.902774559133963}}

In analogy to the `task_dict` which defines the tasks to be executed by the simulation code the `result_dict` summarizes 
the results of the calculations. In this case the energies calculated for the specific strains. By ordering both the 
`task_dict` and the `result_dict` with the same labels, the `EnergyVolumeCurveWorkflow` object is able to match the 
calculation results to the corresponding structure. Finally, in the third step the `analyse_structures()` function takes
the `result_dict` as an input and fits the Equation of State with the fitting parameters defined in the first step:

In [3]:
fit_dict = workflow.analyse_structures(output_dict=result_dict)
fit_dict

{'poly_fit': array([-9.30297838e-05,  2.19434659e-02, -1.68388816e+00,  2.73605421e+01]),
 'fit_type': 'polynomial',
 'fit_order': 3,
 'volume_eq': 66.44252286136938,
 'energy_eq': -14.936703222046024,
 'bulkmodul_eq': 72.38919826515837,
 'b_prime_eq': 4.453836548660183,
 'least_square_error': 4.432974524482514e-09,
 'volume': [63.10861874999998,
  63.77291999999998,
  64.43722124999998,
  65.1015225,
  65.76582375000004,
  66.43012500000002,
  67.09442624999994,
  67.75872750000002,
  68.42302874999999,
  69.08732999999997,
  69.75163125000002],
 'energy': [-14.895378072823371,
  -14.910819737657517,
  -14.922307241120716,
  -14.930392279319857,
  -14.935048569966273,
  -14.93666639636423,
  -14.935212782128978,
  -14.931045138840679,
  -14.924165445707086,
  -14.914703574004657,
  -14.902774559133963]}

The bulk modulus for Aluminium is calculated using the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) simulation code by fitting
the Equation of State with a third order polynomial over a volume range of +/-5% to be 72.3GPa.  

## Elastic Matrix
An alternative approach to calculate the bulk modulus is based on the relation `B = (1/3) (C11 + 2 C12 )`. The bulk
modulus can be calculated based on the sum of the first elastic constant `C11` and twice the second elastic constant `C12`
divided by there. 

In [4]:
from ase.build import bulk
from atomistics.calculators.ase import evaluate_with_ase
from atomistics.workflows.elastic.workflow import ElasticMatrixWorkflow
from gpaw import GPAW, PW

workflow = ElasticMatrixWorkflow(
    structure=bulk("Al", a=4.05, cubic=True),
    num_of_point=5,
    eps_range=0.05,
    sqrt_eta=True,
    fit_order=2
)
task_dict = workflow.generate_structures()
task_dict

{'calc_energy': OrderedDict([('s_e_0',
               Atoms(symbols='Al4', pbc=True, cell=[4.05, 4.05, 4.05])),
              ('s_01_e_m0_05000',
               Atoms(symbols='Al4', pbc=True, cell=[3.8421673571095107, 3.8421673571095107, 3.8421673571095107])),
              ('s_01_e_m0_02500',
               Atoms(symbols='Al4', pbc=True, cell=[3.94745170964797, 3.94745170964797, 3.94745170964797])),
              ('s_01_e_0_02500',
               Atoms(symbols='Al4', pbc=True, cell=[4.150015060213919, 4.150015060213919, 4.150015060213919])),
              ('s_01_e_0_05000',
               Atoms(symbols='Al4', pbc=True, cell=[4.247675835085893, 4.247675835085893, 4.247675835085893])),
              ('s_08_e_m0_05000',
               Atoms(symbols='Al4', pbc=True, cell=[3.8421673571095107, 3.8421673571095107, 4.05])),
              ('s_08_e_m0_02500',
               Atoms(symbols='Al4', pbc=True, cell=[3.94745170964797, 3.94745170964797, 4.05])),
              ('s_08_e_0_02500',
       

In analogy to the example with the `EnergyVolumeCurveWorkflow` above, the `ElasticMatrixWorkflow` is initialized with all
the parameters required to generate the atomistic structures and afterwards fit the resulting energies. By calling the
`generate_structures()` function the task dictionary `task_dict` is generated. The task dictionary specifies that the 
energy should be calculated for a total of thirteen structures with different displacements. In the second step the 
structures are again evaluated with the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) simulation code: 

In [5]:
result_dict = evaluate_with_ase(
    task_dict=task_dict,
    ase_calculator=GPAW(
        xc="PBE",
        mode=PW(300),
        kpts=(3, 3, 3)
    )
)
result_dict


  ___ ___ ___ _ _ _  
 |   |   |_  | | | | 
 | | | | | . | | | | 
 |__ |  _|___|_____|  23.9.1
 |___|_|             

User:   janssen@cmpc06
Date:   Wed Nov 15 11:29:33 2023
Arch:   x86_64
Pid:    15086
CWD:    /home/janssen/notebooks/2023/2023-11-15-atomistics-example
Python: 3.10.13
gpaw:   /home/janssen/mambaforge/lib/python3.10/site-packages/gpaw
_gpaw:  /home/janssen/mambaforge/lib/python3.10/site-packages/
        _gpaw.cpython-310-x86_64-linux-gnu.so
ase:    /home/janssen/mambaforge/lib/python3.10/site-packages/ase (version 3.22.1)
numpy:  /home/janssen/mambaforge/lib/python3.10/site-packages/numpy (version 1.26.0)
scipy:  /home/janssen/mambaforge/lib/python3.10/site-packages/scipy (version 1.11.3)
libxc:  6.2.2
units:  Angstrom and eV
cores: 1
OpenMP: True
OMP_NUM_THREADS: 1

Input parameters:
  kpts: [3 3 3]
  mode: {ecut: 300.0,
         name: pw}
  xc: PBE

System changes: positions, numbers, cell, pbc, initial_charges, initial_magmoms 

Initialize ...

species:
  Al:
    n

{'energy': {'s_e_0': -14.93666639636423,
  's_01_e_m0_05000': -14.509157650668122,
  's_01_e_m0_02500': -14.841982287144095,
  's_01_e_0_02500': -14.861851384196324,
  's_01_e_0_05000': -14.667794842770691,
  's_08_e_m0_05000': -14.761984597147846,
  's_08_e_m0_02500': -14.915410385310373,
  's_08_e_0_02500': -14.906256779097582,
  's_08_e_0_05000': -14.792358225784064,
  's_23_e_m0_05000': -14.27602069468719,
  's_23_e_m0_02500': -14.828566180641806,
  's_23_e_0_02500': -14.919070452536213,
  's_23_e_0_05000': -14.61301941504421}}

The atomistic structures are evaluated with the `evaluate_with_ase()` function, which returns the `result_dict`. This 
`result_dict` in analogy to the `task_dict` contains the same keys as well as the energies calculated with the 
[GPAW](https://wiki.fysik.dtu.dk/gpaw/) simulation code. Finally, the `result_dict` is provided as an input to the 
`analyse_structures()` function to calculate the corresponding elastic constants: 

In [6]:
elastic_dict = workflow.analyse_structures(output_dict=result_dict)
elastic_dict

OrderedDict([('SGN', 225),
             ('v0', 66.43012500000002),
             ('LC', 'CI'),
             ('Lag_strain_list', ['01', '08', '23']),
             ('epss', array([-0.05 , -0.025,  0.   ,  0.025,  0.05 ])),
             ('e0', -14.93666639636423),
             ('strain_energy',
              [[(-0.05, -14.509157650668122),
                (-0.025, -14.841982287144095),
                (0.0, -14.93666639636423),
                (0.02500000000000001, -14.861851384196324),
                (0.05, -14.667794842770691)],
               [(-0.05, -14.761984597147846),
                (-0.025, -14.915410385310373),
                (0.0, -14.93666639636423),
                (0.02500000000000001, -14.906256779097582),
                (0.05, -14.792358225784064)],
               [(-0.05, -14.27602069468719),
                (-0.025, -14.828566180641806),
                (0.0, -14.93666639636423),
                (0.02500000000000001, -14.919070452536213),
                (0.05, -14.61

The bulk modulus calculated from the elastic constants `C11` and `C12` based on a strain of +/- 5% is calculated with 
the [GPAW](https://wiki.fysik.dtu.dk/gpaw/) simulation code to be 74.9GPa. This differs from the bulk modulus calculated
from the Equation of State above by 2.6GPa. In comparison to the experimental bulk modulus for Aluminium which is
[reported to be 76GPa](https://periodictable.com/Elements/013/data.html) the calculation based on the elastic constants
seem to be more precise, still this is more likely related to error cancellation. In general elastic properties calculated
from density functional theory are expected to have errors of about 5-10% unless carefully converged.